# Cellular Automata

In [1]:
# Define play area
using NPZ

include("WGR-MCMClib/WGR.jl")
using .WGR

import Base.flush

using Formatting
using Test

# buf = WGR.BlockBuffer(32 * Threads.nthreads())
buf = WGR.BlockBuffer(128)
setBlocks(x,y,z,b) = WGR.setBlocks!(buf, x, y, z, b)
flush() = WGR.flush!(buf)
refreshNd(a,p,f = false) = WGR.refreshNd!(buf,a,p,f)

refreshNd (generic function with 2 methods)

In [203]:
# Parameters
offset = [0, 65, 0] # origin of the playground
size = [64, 1, 64]

groundPalette = [
    0x1f2f54ff,
    0x465daaff
]

blocks = [
    0,
    0xff0000ff,
    0x444444ff,
]

# Set-up the play area
function ground(offset, size, palette)
    for x in 1:size[1]
        for z in 1:size[3]
            setBlocks(offset[1] + x, offset[2], offset[3] + z, palette[(x + z) % 2 + 1])
            for y in 1:size[2]
                setBlocks(offset[1] + x, offset[2] + y, offset[3] + z, 0)
            end
        end
    end
end

# Ground
groundSize = copy(size)
groundSize[2] = 1
ground(offset, groundSize, groundPalette)

# Template blocks
for b in 1:length(blocks)
    setBlocks(offset[1] + b, offset[2], offset[3], blocks[b])
end

flush()

0

In [206]:
# Read-in
initial_raw = WGR.GetBlockWithin((offset .+ 1)..., (offset + size)...)
function convert(raw, bs)
    res = zero(raw) .% UInt8
    for p in eachindex(raw)
        for bi in eachindex(bs)
            if bs[bi] == raw[p]
                res[p] = bi
            end
        end
    end
    return res
end

initial = convert(initial_raw, blocks)
@show initial

neighbors = [
    [1, 0, 0],
    [1, 0, 1],
    [1, 0,-1],
    [0, 0, 1],
    [0, 0,-1],
    [-1,0, 0],
    [-1,0, 1],
    [-1,0,-1],
]

rule = ones(UInt8, 3, 9, 9, 9)

# Game of life
rule[3, :, :, 3] .= 3
rule[3, :, :, 4] .= 3
rule[1, :, :, 4] .= 3

# Red thingy
rule[3, :, :, 5] .= 2
rule[3, :, 2, 2:6] .= 3
rule[2, :, :, :] .= 1

# Simulate
function simulate(arr)
    narr = copy(arr)
    for rp in CartesianIndices(arr)
        p = [rp[1], rp[2], rp[3]]
        count = [0, 0, 0]
        for n in neighbors
            np = @. mod((p .+ n .- 1), size) .+ 1
            count[arr[np...]] += 1
        end
        count .+= 1
        newp = rule[arr[p...], count...]
        # @show newp
        if narr[p...] != newp
            narr[p...] = newp
            setBlocks((offset .+ p)..., blocks[newp])
        end
    end
    flush()
    return narr
end

for i in 1:1000
    initial = simulate(initial)
end

initial = [0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01;;; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01;;; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0x01; 0